<a href="https://colab.research.google.com/gist/joshpayne/4733493f20f6cb54e20d3ee70ac452a2/quantrank_problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

import numpy as np

from scipy.optimize import minimize, brent

from statsmodels.tsa.stattools import coint

import pylab as plt

from copy import deepcopy as copy


In [8]:
"""1) Generate a series with 1000 daily steps that has a Sharpe of 2+/-0.1

Medium/Hard
"""

a = np.random.randn(1000)

mu = np.mean(a)

sigma = np.std(a)

def f(x,S):

    return np.abs(np.mean(a+x)/np.std(a+x)*16-S)

x = brent(f,args=(2,))

print(f'x={x:.4f}, S={np.mean(a+x)/np.std(a+x)*16:.2f}')

x=0.1190, S=2.00


In [9]:
"""

2) You have an amazing trading strategy that consistently makes 2% per month.

You start with $100 and take out $20 at the beginning of each new year. How

long will it take to become a billionaire?

Medium

"""

x0 = 100

i = 0

while x0<1e9:

    x0*=1.02

    if not i%12:

        x0-=20

    # print(i,x0)

    i+=1

print(f'Years: {np.floor(i/12)}, Months: {i%12}')

Years: 78.0, Months: 11


In [10]:
"""

3) You have a trading strategy with a Sharpe ratio of 1 and a daily vol of 1%.

Calculate the approximate probability of having a drawdown over 10% over 100 days.

Hard

"""

def f(x,S):

    return np.abs(np.mean(a+x)/np.std(a+x)*16-S)

cnt = 0

N = 15000

for i in range(N):

    a = np.random.randn(100)*0.01

    x = brent(f,args=(1,))

    q = np.cumprod(1+a+x)

    # plt.plot(q)

    if any(q<0.9):

        cnt+=1

print(f'Approximate probability of 10% DDWN over 100 days: {cnt/N*100:.2f}%')

Approximate probability of 10% DDWN over 100 days: 2.35%


In [11]:
"""4) Which of these to return series has more profit potential, a or b?

This is about figuring out that the one of the series has autocorrelation
and the other does not.

Medium
"""

a = np.random.randn(1000);

b = -0.2*a[:-1] + a[1:];

plt.plot(np.cumsum(np.sign(-a[:-1])*a[1:]),label='A');

plt.plot(np.cumsum(np.sign(-b[:-1])*b[1:]),label='B');

plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

In [12]:
"""5) Build two time series with <10% correlation and >99% cointegration

The solution is simple but not obvious.

Medium
"""

b = np.sign(np.sin(np.linspace(0,100,1000)))

a = np.sin(np.linspace(0,20,1000))

plt.plot(a,'r',b,'b'); plt.show()

print(coint(a,b)[0], np.corrcoef(a,b)[0,1])

<Figure size 640x480 with 1 Axes>

-10.473127138644246 -0.017535199063983948


In [13]:
"""6) Given the time series bis, create a strategy with an OOS Sharpe larger than 2
All we need to do is to trade AR(1).

Medium
"""

np.random.seed(34)

a = np.random.randn(1000);

b = -0.2*a[:-1] + a[1:];

R = np.sign(-b[:-1])*b[1:]  # THis is the stratgy

bis = b[:600]

Ros = R[600:].mean()/R[600:].std()*16

print(f'OOS Sharpe: {Ros:.2f}')

OOS Sharpe: 2.28


In [14]:
"""7) Optimize this portfolio for the highest Sharpe ratio.

Simple Mean/Var optimization.

Medium
"""

def f(w,a,b):

    R = w[0]*a+w[1]*b

    S = np.mean(R)/np.std(R)*16

    return -S

np.random.seed(32)

a = np.random.randn(1000)*0.02+0.001

b = np.random.randn(1000)*0.005+0.001

plt.plot(np.cumsum(a),'r',np.cumsum(b),'b')

bnds = [(0,1),(0,1)]

const = {'type':'eq','fun':lambda x: sum(x)-1}

res = minimize(f,x0=[0.5,0.5],args=(a,b),bounds=bnds,constraints=const)

print(f'Weights: [{res.x[0]:.2f}, {res.x[1]:.2f}], S: {-res.fun:.2f}')

plt.plot(np.cumsum(a*res.x[0]+b*res.x[1]),c='k',lw=5)

plt.show()

Weights: [0.15, 0.85], S: 2.49


<Figure size 640x480 with 1 Axes>

In [15]:
"""8) We expect an average daily pnl of 1% and a daily std of 1%,

What is the probability of having a negative pnl after 100 days

Easy to Medium
"""

T = 100

N = 100000

a = np.random.randn(T,N)*0.01+0.001

b = np.sum(a,axis=0)

print(f'Prob of Loss after {T} days: {100*len(b[b<0])/N:.0f}%')

Prob of Loss after 100 days: 16%


In [16]:
"""9) What is the optimal leverage to maximize geometric growth for
the given series

Numerical Kelly optimization

Medium
"""

def f(w,a):

    return -np.prod(1+a*w)

np.random.seed(33)

a = np.random.randn(1000)*0.03+0.003

plt.plot(np.cumprod(1+a*0.5)); plt.show()

res = minimize(f, x0=0.1, args=(a,))

print(f'Leverage: {res.x[0]:.3f}')

<Figure size 640x480 with 1 Axes>

Leverage: 3.239


In [17]:
"""

10) We have a series of daily returns s. Build a series with hourly returns

    (24hr day), that closely follows the daily returns and has a Sharpe within

    10% of the daily series.

This is about understanding data and sampling.

Hard
"""

np.random.seed(23)

s = np.random.randn(20)*0.01+0.01

p = np.cumsum(s)

c = np.zeros((len(s)-1)*24+1)

c[::24]=s

raw = copy(c)

for i in range(10):

    np.random.seed(None)

    N = 250000

    a = np.random.randn(24,N)*np.random.rand(24,N)*0.01/np.sqrt(11)

    for i in range(1,len(s)):

        idx = np.argmin(np.abs(sum(a)-(p[i]-p[i-1])))

        c[24*(i-1)+1:24*i+1] = a[:,idx]

    plt.plot(np.cumsum(c))


plt.plot(np.arange(len(s))*24,np.cumsum(s),'ko',ms=10); plt.show()

Sold = np.mean(s)/np.std(s)*16

Snew = np.mean(c)/np.std(c)*np.sqrt(256*24)

print(Sold, Snew, Snew/Sold)

<Figure size 640x480 with 1 Axes>

14.294833363434924 15.103496692351381 1.0565703221826255


In [18]:

"""
11a) Find the optimum parameter combination for a moving average

crossover strategy using numpy only. Each asset is equally weighted.

Parameter optimization

Medium

"""

np.random.seed(12)

a = np.random.randn(1000,3);

b = np.cumsum(1.6*a[:-1,:] + a[1:,:],axis=0);

plt.plot(b); plt.show()

def ma(x,win=20):

    return np.convolve(x,np.ones(win))/win

def MA(x,w):

    return np.apply_along_axis(ma,arr=x,axis=0,win=w)[:x.shape[0],:]

# plt.plot(MA(b,10))

# plt.plot(MA(b,80))

# plt.show()

def backtest(b,w1,w2):

    macross = np.sign(MA(b,w1)-MA(b,w2))

    pnls = macross[:-1,:]*np.diff(b,axis=0)

    pnl = np.sum(pnls,axis=1)

    return np.mean(pnl)/np.std(pnl)*16


res = {}

for w1 in range(10,80):

    for w2 in range(10,80):

        if w1!=w2:

            S = backtest(b,w1,w2)

            res[(w1,w2)] = S

best = max(res.values())

params = [k for k in res if res[k]==best][0]

print(best,params)

<Figure size 640x480 with 1 Axes>

1.5695800743937907 (65, 70)


In [19]:
"""11b) optimize the portfolio of strategies


Optimization of strategy portfolio

Medium/Hard
"""

def raw_backtest(b,w1,w2):

    macross = np.sign(MA(b,w1)-MA(b,w2))

    pnls = macross[:-1,:]*np.diff(b,axis=0)

    return pnls

def f(w,pnls):

    R = np.dot(pnls,w)

    S = np.mean(R)/np.std(R)*16

    return -S

pnls = raw_backtest(b,*params)


bnds = [(0,1) for _ in range(b.shape[1])]

const = {'type':'eq','fun':lambda x: sum(x)-1}

res = minimize(f,x0=np.ones(b.shape[1])/b.shape[1],args=(pnls),bounds=bnds,constraints=const)

print(f'Optimized S: {-res.fun:.2f}')

Optimized S: 1.95


In [20]:
"""

12) We ran some numerical tests and get the following point cloud

as a result. The first two columns of q give the coordinates

of the points and the next two columns hold the parameters that

the function used to produce these points.

We can see that there may be two overlapping domains in this

dataset. Use sklearn to find the regression slopes for the two

domains.

This is about understanding and dealing with the structure of data

Hard

"""

from sklearn.cluster import KMeans

# Creating the dataset

N = 5000

x1=np.random.randn(N); y1=1.4*x1+np.random.randn(N)

x2=np.random.randn(N); y2=-2.7*x2-np.random.randn(N)

params1=(np.random.rand(N,2)*10000).astype(int)

params2=(np.random.rand(N,2)*10000-15000).astype(int)

plt.plot(x1,y1,'bo')

plt.plot(x2,y2,'bo')

plt.show()

m1 = np.polyfit(x1,y1,1)

m2 = np.polyfit(x2,y2,1)

orig_slopes = [np.round(m1[0],3),np.round(m2[0],3)]

pset1 = np.vstack([x1,y1,params1[:,0],params1[:,1]]).T

pset2 = np.vstack([x2,y2,params2[:,0],params2[:,1]]).T

q = np.vstack([pset1,pset2])

np.random.shuffle(q)


# solving the exercise

model = KMeans(n_clusters=2)

model.fit(q)

labels = model.labels_

ax=[]; bx=[]; ay=[]; by=[]

for i,label in enumerate(labels):

    if label==0:

        # plt.plot(q[i,0],q[i,1],'ro')

        ax.append(q[i,0])

        ay.append(q[i,1])

    else:

        # plt.plot(q[i,0],q[i,1],'bo')

        bx.append(q[i,0])

        by.append(q[i,1])

ma = np.polyfit(ax,ay,1)

mb = np.polyfit(bx,by,1)

slopes=[ma[0],mb[0]]

print(f'Correct Solution for cluster 1: {np.round(slopes[0],3) in orig_slopes}')

print(f'Correct Solution for cluster 2: {np.round(slopes[1],3) in orig_slopes}')

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
'''13) An unknown function gives a poor linear fit. What can we do

find a better way of fitting this function

Using ranking to linearize data sets

Easy to Medium
'''

from sklearn.linear_model import LinearRegression

N = 500

x = np.linspace(0,1,N)

y = np.exp(-8/x)+np.random.randn(N)*0.000001

plt.plot(x,y,'o')

model = LinearRegression()

model.fit(np.array([x]).T,y)

R2_orig = model.score(np.array([x]).T,y)

print(R2_orig)

# Solving the problem

model.fit(np.array([np.argsort(x)]).T,np.argsort(y))

R2 = model.score(np.array([np.argsort(x)]).T,np.argsort(y))

print(R2)

print(f'Problem solved: {R2>R2_orig+0.05}')

In [ ]:
"""
14)
An exotic derivative behaves like a call option with the exception that it automatically
expires worthless if the close price drops below a certain barrier. Using MC, calculate
the value of such an option with a strike of $100, a spot of 95$, an IV of 30% and
a barrier of $92, with an expiry of 1year. The risk-free rate is assumed to be zero.
(256 trading days per year)
"""

"""A) Vanilla Option - Medium"""
spot = 95
strike = 100
IV = 0.25
barrier = 90
r = 0.16

N = 700000



daily_sigma = IV / np.sqrt(256)

r_daily = (1+r)**(1.0/256)-1

paths = np.random.randn(N,256)*daily_sigma

prices = ((np.cumprod(1+paths, axis=1))*spot)

final = prices[:,-1]

vanilla_price = np.sum((final-strike)[final>strike])/N

print(f'Vanilla Price: {vanilla_price:.4f}')


"""B) Barrier option calcs - Medium"""


itm_idxs = np.argwhere(final>strike)

min_prices  = np.min(prices,axis=1)

not_out_idxs = np.argwhere(min_prices>barrier)


survivors = list(set(itm_idxs.flatten()).intersection(set(not_out_idxs.flatten())))

barrier_price = np.sum((final-strike)[survivors])/N
print(f'Barrier Price: {barrier_price:.2f}')

analytic_solution = 7.39
print(f'Solution within 1% of analytical: {np.abs(vanilla_price-analytic_solution)-1<0.01}')


In [ ]:
"""
15)
An exotic derivative behaves like a call option with the exception that it automatically
expires worthless if the close price drops below a certain barrier. Using MC, calculate
the value of such an option with a strike of $100, a spot of 95$, an IV of 30% and
a barrier of $92, with an expiry of 1year. Now we include a non-zero risk-free rate of 5%.
(256 trading days per year)

Hard
"""

"""A) Vanilla Option"""
spot = 95
strike = 100
IV = 0.25
barrier = 90

rf = 0.05

N = 700000

daily_sigma = IV / np.sqrt(256)

r_daily = (1+rf)**(1.0/256)-1
rx = np.cumprod(1+np.ones((N,256))*r_daily,axis=1)-1

paths = np.random.randn(N,256)*daily_sigma

prices = (((np.cumprod(1+paths, axis=1))+rx)*spot)

final = prices[:,-1]

vanilla_price = np.sum((final-strike)[final>strike])/N

print(f'Vanilla Price: {vanilla_price:.4f}')


"""B) Barrier option calcs"""


itm_idxs = np.argwhere(final>strike)

min_prices  = np.min(prices,axis=1)

not_out_idxs = np.argwhere(min_prices>barrier)


survivors = list(set(itm_idxs.flatten()).intersection(set(not_out_idxs.flatten())))

barrier_price = np.sum((final-strike)[survivors])/N
print(f'Barrier Price: {barrier_price:.2f}')

analytic_solution = 9.38
print(f'Solution within 1% of analytical: {np.abs(vanilla_price-analytic_solution)-1<0.01}')

In [ ]:

"""
16)

Build a profitable strategy by ranking the normalized price levels of the given stocks each day

and trade the top-5 and bottom-5. What is the sharpe ratio of the resulting strategy?

"""

np.random.seed(12)

M = 20

band = 5

gamma = np.random.rand(M)

r = np.random.randn(1001,M)

r = r[:-1,:]*gamma + r[1:]

p = np.cumsum(r,axis=0)

p = p - np.mean(p,axis=0)

plt.plot(p,alpha=0.5)

ranks = np.argsort(p,axis=1)

upranks = copy(ranks)

upranks[upranks<M-band] = 0

upranks = np.sign(upranks)

uppnl = -np.sum(upranks[:-1,:]*r[1:,:],axis=1)

plt.plot(np.cumsum(uppnl),lw=5,alpha=0.6)

downranks = copy(ranks)

downranks[downranks>band] = 0

downranks = np.sign(downranks)

downpnl = -np.sum(downranks[:-1,:]*r[1:,:],axis=1)

plt.plot(np.cumsum(downpnl),lw=5,alpha=0.6)

R = uppnl+downpnl

plt.plot(np.cumsum(R/2),lw=5,c='k')

S = np.mean(R)/np.std(R)*16

print(f'Problem Solved: {np.round(S,2)==1.53}')

In [ ]:
"""17) Given a Sharpe ratio of 2, what is the max expected drawdown for a 1000d period?"""

q = []

for _ in range(10000):

    a = np.random.randn(1000)*0.01

    mu = np.mean(a)

    sigma = np.std(a)

    def f(x,S):

        return np.abs(np.mean(a+x)/np.std(a+x)*16-S)

    x = brent(f,args=(2,))

    q.append(x+a)

cumpnls = np.cumprod(1+np.array(q).T,axis=0)

maximums = np.maximum.accumulate(cumpnls,axis=0)

av_max_ddwn = np.mean(np.max(maximums/cumpnls-1,axis=0))

print(f'Aver Max DDWN: {av_max_ddwn*100:.2f}%')

print(f'Problem Solved: {np.round(av_max_ddwn,2)==0.17}')




In [ ]:
"""18)
For a strategy with expected Sharpe=1.0 and annual vol of 16%,

how much income would the strategy generate if we maintain

an annual strategy profit of 10%, (Sharpe is normally distributed with sigma=1)"""

q = []

for _ in range(10000):

    a = np.random.randn(256)*0.01

    mu = np.mean(a)

    sigma = np.std(a)

    def f(x,S):

        return np.abs(np.mean(a+x)/np.std(a+x)*16-S)

    x = brent(f,args=(np.random.randn()+1,))

    q.append(x+a)

pnls = np.prod(1+np.array(q).T,axis=0)

print(f'Ann Income: {(np.mean(pnls-1)-0.1)*100:.2f}%')

print(f'Problem Solved: {np.round((np.mean(pnls-1)-0.1),2)==0.07}')


In [ ]:
"""Subtract a value lambda from the off-diagonals of a matrix and leave the
diagonals as they are without using loops (i.e. needs to be fast)"""

mat = np.random.randn(7000,7000)
lmbda = 0.1

diag = np.eye(mat.shape[0])*(np.diag(mat))
off_diag = (mat-diag)
result = off_diag-lmbda+np.eye(mat.shape[0])*lmbda+diag
assert(result[2134,347]==mat[2134,347]-0.1)
assert(result[1865,1865]==mat[1865,1865])
print('Problem Solved!')

In [ ]:
"""
n is an integer that represents different market states.
Calculate the transition probabilities between different states in a state vector
l and put the probabilities in a NxN matrix where N is the number of unique states.
"""

l = np.random.randint(0,3,1000)
l[999]=3
q = np.unique(l)
m = np.zeros((len(q),len(q)))
for i in range(1,len(l)):
  m[l[i-1],l[i]]+=1
result  = m/(len(l)-1)
assert(sum(result[3,:])==0)
print('Problem Solved!')

In [ ]:
"""Given an array of daily returns for N trading strategies, find the average drawdown period (no Pandas allowed)
Run in less than 10ms for extra brownie points.
"""
np.random.seed(16)
def dd_len(r):
  p = np.cumsum(r,axis=0)
  rmax = np.maximum.accumulate(p)
  # plt.plot(p)
  # plt.plot(rmax)
  ddwn = rmax-p
  # plt.plot(ddwn)
  diffs = np.sign(ddwn)
  diffdiff = np.diff(diffs,axis=0)
  n_ddwns = np.ceil(np.sum(np.abs(diffdiff)/2,axis=0))
  return np.mean(np.sum(diffs,axis=0)/n_ddwns)

def test():
  r = np.zeros((1000,2))
  r[30,0]=-1; r[70,0]=1; r[130,0]=-1; r[180,0]=1
  r[60,1]=-1; r[120,1]=1; r[130,1]=-1; r[140,1]=1
  assert(dd_len(r)==40)
  print('Test passed!')

test()

r = np.random.randn(1000,200)
res = dd_len(r)
%timeit dd_len(r)
assert(np.round(res,2)==108.72)
print('Problem Solved!')

In [ ]:
"""
Find the time between the minima of the two largest drawdowns.
Note, a drawdown is defined as the time between two break-even points.
"""

np.random.seed(27)
r = np.random.randn(5000,2)
p = np.cumsum(r,axis=0)
rmax = np.maximum.accumulate(p)
# plt.plot(p)

ddwn = rmax-p
plt.plot(ddwn)
# plt.grid()

a=np.argsort(ddwn,axis=0).argsort(axis=0)
b=np.sign(ddwn)
c = np.argsort(ddwn,axis=0)
# find the two largest indices with a zero in-between
# plt.plot(a*b)
for j in range(r.shape[1]):
  for i in range(2,len(ddwn)):
    k = min(ddwn[min([c[-1,j],c[-i,j]]):max([c[-1,j],c[-i,j]])][:,j])
    if k==0:
      break

  print(np.abs(c[-1,j]-c[-i,j]))

In [ ]:

for j in range(ddwn.shape[1]):
  ddwns = []
  t = []
  dd = 0
  for i,d in enumerate(ddwn[1:,j]):
    if d>dd:
      dd = d
      # print(dd)

    if ((d==0) or i==ddwn.shape[0]) and ddwn[i-1,j]>0:
      # print('--->',dd)
      ddwns.append(dd)
      t.append(i)
      dd = 0
  # print(ddwns)
  idx = np.argsort(ddwns)[-2:]
  dist = np.abs(np.diff(np.array(t)[idx]))
  print(dist)

In [ ]:
plt.plot(ddwn)


In [ ]:
"""Given a surface, how many steps is the shortest distance from the origin to
the highest point?"""